In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import metrics
import pandas as pd
import os
from PIL import Image as im
import time
import gradio as gr

In [2]:
apple_dir = r"plant_dataset_2\apple"
corn_dir = r"plant_dataset_2\corn"
potato_dir = r"plant_dataset_2\potato"
tomato_dir = r"plant_dataset_2\tomato"

class_dic = {
    0:'apple',
    1:'corn',
    2:'potato',
    3:'tomato'
}

apple_class_dic = {
    0: 'Apple___Apple_scab',
    1: 'Apple___Black_rot',
    2: 'Apple___Cedar_apple_rust',
    3: 'Apple___healthy'
}

corn_class_dic = {
    0: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 
    1: 'Corn_(maize)___Common_rust_', 
    2: 'Corn_(maize)___Northern_Leaf_Blight', 
    3: 'Corn_(maize)___healthy'
}

potato_class_dic = {
    0: 'Potato___Early_blight', 
    1: 'Potato___Late_blight', 
    2: 'Potato___healthy'
}

tomato_class_dic = {
    0: 'Tomato___Bacterial_spot',
    1: 'Tomato___Early_blight', 
    2: 'Tomato___Late_blight', 
    3: 'Tomato___Leaf_Mold', 
    4: 'Tomato___healthy'
}
img_height = 256
img_width = 256


In [3]:
df = pd.read_csv(r"disease_class.csv")

df.head()

,class,class_id
0,Apple___Apple_scab,0
1,Apple___Black_rot,1
2,Apple___Cedar_apple_rust,2
3,Apple___healthy,3
4,Corn_(maize)___Cercospora_leaf_spot Gray_leaf_...,4


In [4]:
#TO REMOVE WARNING MESSAGE
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


plant_classify_model = tf.keras.models.load_model('saved_model/plant_classify_model')
apple_model = tf.keras.models.load_model('saved_model/apple_model')
corn_model = tf.keras.models.load_model('saved_model/corn_model')
potato_model = tf.keras.models.load_model('saved_model/potato_model')
tomato_model = tf.keras.models.load_model('saved_model/tomato_model')

In [5]:
article = r'''
<table>
<tr>
<th>No</th>
<th>Target Plant</th>
<th>Types Of Disease</th>
</tr>

<tr>
<td rowspan="3">1</td>
<td rowspan="3">Apple</td>
<td>Apple Scab</td>
</tr>
    
<tr>
<td>Apple BLack Rust</td>
</tr>
    
<tr>
<td>Cedar Apple Rust</td>
</tr>

<tr>
<td rowspan="3">2</td>
<td rowspan="3">Corn</td>
<td>Cercospora Leaf Spot/Grey Leaf Spot</td>
</tr>
    
<tr>
<td>Common Rust</td>
</tr>
    
<tr>
<td>Northern Corn Leaf Blight</td>
</tr>
<tr>
<td rowspan="2">3</td>
<td rowspan="2">Potato</td>
<td>Early Blight</td>
</tr>
    
<tr>
<td>Late Blight</td>
</tr>

<tr>
<td rowspan="4">4</td>
<td rowspan="4">Tomato</td>
<td>Early Blight</td>
</tr>
    
<tr>
<td>Late Blight</td>
</tr>    

<tr>
<td>Bacterial Spot</td>
</tr> 
<tr>
<td>Leaf Mold</td>
</tr> 
</table>
'''

In [6]:
def predfunc(img1):

    
    if isinstance(img1, np.ndarray):
        img1 = im.fromarray(np.uint8((img1)))
        
    
    img = img1.copy()
    #print(type(img))
    #print(img.size)
    #print()
    
    img_array = tf.keras.utils.img_to_array(img)
    data = im.fromarray((img_array).astype(np.uint8))
    data.save('input_image.png')
    
    img = img.resize((256, 256))
    
    img_array = tf.keras.utils.img_to_array(img)
    
    #print(type(img_array))
    #print(img_array.shape)
    #print()
    
    gray_img_array = tf.image.rgb_to_grayscale(img_array)    
    gray_img_array = tf.expand_dims(gray_img_array, 0)
    
    
    img_array = tf.expand_dims(img_array, 0)
    #print(type(img_array))
    #print(img_array.shape)
    
    start = time.time()
    pred1 = plant_classify_model.predict(gray_img_array)
    
    score1 = tf.nn.softmax(pred1)
    
    if np.argmax(score1)==0:
        pred2 = apple_model.predict(img_array)
    elif np.argmax(score1)==1:
        pred2 = corn_model.predict(img_array)
    elif np.argmax(score1)==2:
        pred2 = potato_model.predict(img_array)
    else:
        pred2 = tomato_model.predict(img_array)

    score2 = tf.nn.softmax(pred2)
    
    end = time.time()
    pred_time = end-start
    
    ans_dic = {}
    
    if np.argmax(score1)==0:
        
        name =   apple_class_dic[np.argmax(score2)]
        
        for i in range(len(apple_class_dic)):
            ans_dic[apple_class_dic[i]] = np.array(score2)[0][i]*1
        
    elif np.argmax(score1)==1:
        name = corn_class_dic[np.argmax(score2)]
        
        for i in range(len(corn_class_dic)):
            ans_dic[corn_class_dic[i]] = np.array(score2)[0][i]*1
        
    elif np.argmax(score1)==2:
        name = potato_class_dic[np.argmax(score2)]
        
        for i in range(len(potato_class_dic)):
            ans_dic[potato_class_dic[i]] = np.array(score2)[0][i]*1
        
    else:        
        name = tomato_class_dic[np.argmax(score2)]
        
        for i in range(len(tomato_class_dic)):
            ans_dic[tomato_class_dic[i]] = np.array(score2)[0][i]*1
    
    c_id = int(df['class_id'][df['class'] == name])
    
    pred_time = str(round(pred_time, 4)*100)+' ms'
    
    
    return [ans_dic, pred_time, c_id]




img_path = r"plant_dataset_3\corn\Corn_(maize)___healthy\0b03beed-0247-4d38-87ba-eb5b94415312___R.S_HL 8233 copy 2.jpg"


img = load_img(img_path)

predicted = predfunc(img)


print(max(predicted[0]))

print(predicted[1])


1/1 [==============================] - 0s 68ms/step
Corn_(maize)___healthy
523.88 ms


In [7]:
def img_transform(e):
    
    size = 256
    img = r"input_image.png"
    inp = load_img(img, target_size=(size, size))
    
    new_size = str(inp.size)  #take size of new image
    
    img_array = tf.keras.utils.img_to_array(inp)
    
    g_img = im.open(img).convert('L')
    
    orig_size = str(g_img.size) #take size of original image
    
    print(orig_size)
    print(new_size)
    siz = img_array.shape
    
    dim = siz[2]
    a = tf.expand_dims(img_array, 0)
    
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), input_shape=(size, size, dim)))
    model.compile('adam', 'mean_squared_error')
    pred = model.predict(a, steps=1)
    array = np.reshape(pred, (int(size/2), int(size/2), dim))
    
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.compile('adam', 'mean_squard_error')
    pred1 = model.predict(a, steps=1)
    array1 = np.reshape(pred1, (int(size/4), int(size/4), dim))
    
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.compile('adam', 'mean_squard_error')
    pred2 = model.predict(a, steps=1)
    array2 = np.reshape(pred2, (int(size/8), int(size/8), dim))
    
    
    array = np.uint8(array)
    print(array.shape)
    output = im.fromarray(array)
    
    array1 = np.uint8(array1)
    output1 = im.fromarray(array1)
    
    array2 = np.uint8(array2)
    output2 = im.fromarray(array2)
    
    return [output, output1, output2, inp, g_img, orig_size, new_size] #pool1, pool2, pool3, input, gray, orig_size, new_size

paths = r"testing files\Potato___Early_blight\0F83C659-0115-466C-8EA2-B4E05BC0C089.jpeg"
img_transform(paths)

(256, 256)
(256, 256)
1/1 [==============================] - 0s 27ms/step
(128, 128, 3)


[<PIL.Image.Image image mode=RGB size=128x128>,
 <PIL.Image.Image image mode=RGB size=64x64>,
 <PIL.Image.Image image mode=RGB size=32x32>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x256>,
 <PIL.Image.Image image mode=L size=256x256>,
 '(256, 256)',
 '(256, 256)']

In [8]:
drop_out = '''
<br>

<table>
            <h1>With Dropout Layer (CNN Max Pooling is used)</h1>
            <tr>
                <th>Model</th>
                <th>Best Validation Accuracy</th>
                <th>Valid-Sample Number</th>
                <th>Time Taken To Train Per Batch</th>
            </tr>
            <tr>
                <td>Plant type</td>
                <td>0.9602</td>
                <td>290</td>
                <td>18ms</td>
            </tr>
            <tr>
                <td>Apple</td>
                <td>0.9812</td>
                <td>75</td>
                <td>22ms</td>
            </tr>
            <tr>
                <td>Corn</td>
                <td>0.963</td>
                <td>71</td>
                <td>23ms</td>
            </tr>
            <tr>
                <td>Potato</td>
                <td>0.9674</td>
                <td>55</td>
                <td>22ms</td>
            </tr>
            <tr>
                <td>Tomato</td>
                <td>0.9437</td>
                <td>90</td>
                <td>25ms</td>
            </tr>
        </table>
'''

no_drop_out = '''
<br>

<table>
            <h1>Without Dropout Layer (CNN Max Pooling is used)</h1>
            <tr>
                <th>Model</th>
                <th>Best Validation Accuracy</th>
                <th>Valid-Sample Number</th>
                <th>Time Taken To Train Per</th>
            </tr>
            <tr>
                <td>Plant type</td>
                <td>0.9338</td>
                <td>290</td>
                <td>15ms</td>
            </tr>
            <tr>
                <td>Apple</td>
                <td>0.9418</td>
                <td>75</td>
                <td>24ms</td>
            </tr>
            <tr>
                <td>Corn</td>
                <td>0.9575</td>
                <td>71</td>
                <td>21ms</td>
            </tr>
            <tr>
                <td>Potato</td>
                <td>0.9868</td>
                <td>55</td>
                <td>21ms</td>
            </tr>
            <tr>
                <td>Tomato</td>
                <td>0.8754</td>
                <td>90</td>
                <td>23ms</td>
            </tr>
        </table>
'''

md_dt = '''
    <br>The plant leaf disease classification system is trained with 21612 leaf-photos and is validated with 9261 leaf-photos.
    <br>For now, the system can detect 12 diseases of 4 classes of plant. The detection system is trained with neural netwroks.
    The image features are extracted using Convolutional Neural Network Algorithm. <br>The system contains 5 models, 1 is to classify the class of the plant and the rests are to classify their respective plant leaf diseases.
    
'''

avg_pooling_table = '''

    <table>
        <tr>
            <th></th>
            <th>Best Validation accuracy</th>
            <th>Time Taken to train Per batch</th>
        </tr>
        <tr>
            <td>Plant type</td>
            <td>0.9656</td>
            <td>40ms</td>
        </tr>
        <tr>
            <td>Apple</td>
            <td>0.9833</td>
            <td>51ms</td>
        </tr>
        <tr>
            <td>Corn</td>
            <td>0.9599</td>
            <td>51ms</td>
        </tr>
        <tr>
            <td>Potato</td>
            <td>0.9846</td>
            <td>52ms</td>
        </tr>
        <tr>
            <td>Tomato</td>
            <td>0.9678</td>
            <td>53ms</td>
        </tr>
    </table>
'''
model_summary = '''

<table>
        <tr>
            <th>Layer ( type )</th>
            <th>Output Shape</th>
            <th>Param #</th>
        </tr>
        <tr>
            <td>resizing_1 (Resizing)</td>
            <td>(None,256,256,3)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>rescaling_1 (Rescaling)</td>
            <td>(None,256,256,3)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>random_rotation_1 (Random Rotation)</td>
            <td>(None,256,256,3)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>conv2d_3 (Conv 2D)</td>
            <td>(None,256,256,16)</td>
            <td>448</td>
        </tr>
        <tr>
            <td>max_pooling2d_3 (MaxPooling 2D)</td>
            <td>(None,128,128,16)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>conv2d_4 (Conv 2D)</td>
            <td>(None,128,128,32)</td>
            <td>4640</td>
        </tr>
        <tr>
            <td>max_pooling2d_4 (MaxPooling 2D)</td>
            <td>(None,64,64,32)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>conv2d_5 (Conv 2D)</td>
            <td>(None,64,64,64)</td>
            <td>18496</td>
        </tr>
        <tr>
            <td>max_pooling2d_5 (MaxPooling 2D)</td>
            <td>(None,32,32,64)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>flatten_1 (Flatten)</td>
            <td>(None, 65536)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>dense_3 (Dense)</td>
            <td>(None, 512)</td>
            <td>33554944</td>
        </tr>
        <tr>
            <td>dropout_2 (Dropout)</td>
            <td>(None, 512)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>dense_4 (Dense)</td>
            <td>(None, 256)</td>
            <td>131328</td>
        </tr>
        <tr>
            <td>dropout_3 (Dropout)</td>
            <td>(None, 256)</td>
            <td>0</td>
        </tr>
        <tr>
            <td>dense_5 (Dense)</td>
            <td>(None, 4)</td>
            <td>1028</td>
        </tr>
    </table>
'''

In [9]:



with gr.Blocks() as demo:
    
    with gr.Row():
        
        with gr.Column():
    
            gr.Markdown(
                """
                <h1>Plant Leaf Disease Detection System</h1>
                """
            )
        with gr.Column():
            gr.Markdown(' ')
            
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
        with gr.Column():
            gr.Markdown('# Image acquisition')
            img_input = gr.Image()
        with gr.Column():
            gr.Markdown(' ')
            
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
        with gr.Column():
            pred_btn = gr.Button("Submit")
        with gr.Column():
            gr.Markdown(' ')
            
    with gr.Row():
        with gr.Column():
            pred_time = gr.Textbox(label="Time taken to predict")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
    
    with gr.Row():
        with gr.Column():
            tran_btn = gr.Button('See Inside')
            gr.Markdown(' ')
            
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
        with gr.Column():
            gr.Markdown('# Reshape image into (256, 256) dimension')
        with gr.Column():
            gr.Markdown(' ')
            
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
        with gr.Column():
            orig_size = gr.Textbox(label="Original image size")
            new_size = gr.Textbox(label="New image size")
        with gr.Column():
            gr.Markdown(' ')
            
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
        with gr.Column():
            gr.Markdown('# Gray Scale Image')
        with gr.Column():
            gr.Markdown(' ')
            
    with gr.Row():
        with gr.Column():
            gr.Markdown('''
                  <br>  Image is transformed to gray scale for plant type classification model.
                    Plant type classification is based on shape so it doesn't need color to classify.
                    <br>Grayscale also simplifies the algorithm and reduces computational requirements.
                     
            ''')
        with gr.Column():
            g_img = gr.Image()
        with gr.Column():
            gr.Markdown(" ")
        
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
        with gr.Column():
            gr.Markdown('# <br> Feature Extraction')
        with gr.Column():
            gr.Markdown(' ')
    with gr.Row():
        with gr.Column():
            gr.Markdown("""
            
              <br>  During CNN process, the image is passed through pooling layer to reduce the size of the photo while maintaining its information.
                
            """)
    with gr.Row():
        with gr.Column():
            orig_img = gr.Image(label="image size (256, 256)")
        with gr.Column():
            transformed_img = gr.Image(label="image size after pooling (128, 128)")
        with gr.Column():
            transformed_img2 = gr.Image(label="image size after pooling again(64, 64)")
        with gr.Column():
            transformed_img3 = gr.Image(label="image size after pooling again(32, 32)")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
        with gr.Column():
            gr.Markdown('# <br>Classification result')
        with gr.Column():
            gr.Markdown(' ')
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
        with gr.Column():
            output_label = gr.Label()
        with gr.Column():
            gr.Markdown(' ')
            
    with gr.Row():
        with gr.Column():
            gr.Markdown("# Model detail")
            
    with gr.Row():
        with gr.Column():
            gr.Markdown(md_dt)
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("<br>The following figure is the summary of system model.")
            gr.Markdown("# Step by step process")
            
    with gr.Row():
        with gr.Column():
            gr.Markdown(model_summary)
        with gr.Column():
            gr.Markdown('')
        
    
    with gr.Row():
        with gr.Column():
            gr.Markdown(" # <br>Classification result using average pooling")
            
    with gr.Row():
        with gr.Column():
            gr.Markdown(avg_pooling_table)
        with gr.Column():
            gr.Markdown('''
              <br><br>Overall accuracy ~=0.91</br>
                (evaluated with 96 photos)
            ''')
    
    with gr.Row():
        with gr.Column():
            gr.Markdown('''
                <br>Average pooling method smooths out the image and hence the sharp features may not be identified when this pooling method is used.\n
                Max pooling selects the brighter pixels from the image. It is useful when the background of the image is dark.\n
                Most photos' backgrounds from training dataset are dim, therefore max pooling will be used. 
            ''')
    with gr.Row():
        with gr.Column():
            gr.Markdown('''
               <br>Drop out layer is added to model. Eventhough the model took longer time to train, it has better validation accuracy.
            
            ''')
        
    with gr.Row():
        with gr.Column():
            gr.Markdown(drop_out)
        with gr.Column():
            gr.Markdown('''
                 <br><br><br>Over all accuracy ~= 0.9278</br>
                 (evaluated with 96 photos)
            ''')
    with gr.Row():
        with gr.Column():
            gr.Markdown(no_drop_out)
        with gr.Column():
            gr.Markdown('''
                <br><br><br>Over all accuracy ~= 0.8969</br>
                (evaluated with 96 photos)
            ''')
    
    with gr.Row():
        with gr.Column():
            gr.Markdown(' ')
    with gr.Row():
        with gr.Column():
            gr.Markdown(article)
        with gr.Column():
            gr.Markdown('    ')
    
    
    
    
    with gr.Row():
        with gr.Column():
            t1 = gr.Radio()
        with gr.Column():
            c_id = gr.Textbox(label="Class number")
        
    
    
    pred_btn.click(predfunc, inputs=img_input, outputs=[output_label, pred_time, c_id])
    tran_btn.click(img_transform, inputs=t1, outputs=[transformed_img, transformed_img2, transformed_img3, orig_img, g_img, orig_size, new_size])
    
demo.launch()


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x217618a2970>, 'http://127.0.0.1:7860/', None)

In [10]:
y_test = []

path = r"testing files"
test_list = os.listdir(path)
for i in range(len(test_list)):
    
    img_path = os.path.join(os.getcwd(), 'testing files', test_list[i])
    img_list = os.listdir(img_path)
    
    for j in range(len(img_list)):
        uuu = [int(df['class_id'][df['class']==test_list[i]])]
        y_test.append(uuu)
       

y_predic = []

for i in range(len(test_list)):
    
    img_path = os.path.join(os.getcwd(), 'testing files', test_list[i])
    img_list = os.listdir(img_path)
    for j in range(len(img_list)):
        p = os.path.join(img_path, img_list[j])
        img = load_img(p, target_size=(256, 256))
        u, tim, c_id = predfunc(img)
        y_predic.append([c_id])

m = metrics.Accuracy()
m.update_state(y_test, y_predic)
np.array(m.result())

1/1 [==============================] - 0s 13ms/step


array(0.92783505, dtype=float32)

In [11]:
print(y_test)
print(y_predic)

[[0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [2], [2], [2], [2], [2], [2], [3], [3], [3], [3], [3], [3], [4], [4], [4], [4], [4], [4], [5], [5], [5], [5], [5], [5], [7], [7], [7], [7], [7], [7], [6], [6], [6], [6], [6], [6], [8], [8], [8], [8], [8], [8], [10], [10], [10], [10], [10], [10], [9], [9], [9], [9], [9], [9], [11], [11], [11], [11], [11], [11], [12], [12], [12], [12], [12], [12], [15], [15], [15], [15], [15], [15], [13], [13], [13], [13], [13], [13], [14], [14], [14], [14], [14], [14]]
[[0], [0], [0], [15], [0], [0], [0], [1], [1], [1], [12], [1], [1], [2], [2], [2], [2], [2], [2], [3], [3], [3], [3], [0], [3], [4], [4], [4], [4], [4], [4], [5], [5], [5], [5], [5], [5], [7], [7], [7], [7], [7], [7], [6], [6], [6], [6], [6], [4], [8], [8], [8], [8], [8], [8], [10], [10], [10], [10], [10], [10], [9], [9], [9], [8], [9], [9], [11], [11], [11], [11], [11], [11], [12], [12], [12], [15], [12], [12], [15], [15], [0], [15], [15], [15], [13], [13], [13], [13], [13

In [12]:
m = metrics.Accuracy()
m.update_state(y_test, y_predic)
np.array(m.result())


array(0.92783505, dtype=float32)